In [3]:
import numpy as np
import pandas as pd
import copy
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
x=data.data
y=data.target
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42)

In [4]:
X = np.reshape(x_train,(x_train.shape[0],1,x_train.shape[1]))
y = np.reshape(y_train,(y_train.shape[0],1,1))

In [5]:
def d_logloss(y, a):
    return ((a - y)/(a*(1 - a)))

def d_computeCost(h,Y,m):
    loss = np.square(h-Y)
    cost = -1/(4*m*loss)
    return cost

def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

In [6]:
class Layer:
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        # dBias = output_error

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

In [7]:
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

In [19]:
class Network:
    def __init__(self):
        self.loss = None
        self.loss_prime = None
        self.layers = []
        self.costs = []
        self.iterations = []

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)
                #print(y_train[j], output)
                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)
                #print(err)

                # backward propagation
                error = self.loss_prime(x_train[j],y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            self.costs.append(err)
            self.iterations.append(i)
            #if((i+1) % ((epochs)/4)  == 0 ):
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))
    
    def plot(self,figsize=(7,5)):
        #plt.figure(figsize=figsize)
        plt.plot(self.iterations,self.costs)
        
    def score(self,X,y):
        return 1-(np.sum(((y-self.predict(X))**2))/np.sum((y-np.mean(y))**2))

In [24]:
#activation functions
def relu(x):
    return(np.maximum(0, x))

def relu_prime(x):
    return np.where(x > 0, 1.0, 0.0)

def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return (1 - np.square(np.tanh(x)))

def sigmoid(x):
    #print(x)
    return (1/(1 + np.exp(-x)))

def d_sigmoid(x):
    return ((1 - sigmoid(x)) * sigmoid(x))

def classification_sigmoid(x):
    out = (1/(1 + np.exp(-x)))
    return([1 if p > 0.5 else 0 for p in out])

# Loss Functions 
def logloss(y, a):
    return (-(y*np.log(a) + (1-y)*np.log(1-a)))

def d_logloss(x,y, a):
    return np.dot(x.T,(a - y))

In [25]:
net3 = Network()
net3.add(Layer(30, 30))
net3.add(ActivationLayer(tanh, tanh_prime))
net3.add(Layer(30, 1))
net3.add(ActivationLayer(sigmoid, d_sigmoid))


# train
net3.use(logloss, d_logloss)
net3.fit(X, y, epochs=1000, learning_rate=1e-5)
#net3.score(X,y)

ValueError: shapes (30,1) and (30,1) not aligned: 1 (dim 1) != 30 (dim 0)